# Colored MNIST (Temping to put more value on colors instead of the shape of the digits)
The idea is to create a model, which should be able to asses, if the digit in the image is a low or a high number. The image dataset of colored digits is divided into three parts namely the train, validation and test datasets. In the train and validatin datasets, the low numbers are colored in red and the high numbers are colored in green. In the test dataset, the colors are random. If the model is able to recognize the value of the digits from it's shape, the performance should be nearly equal as the performance on the train and validation datasets. The hypothesis is, that the model will learn to separate low from high digits based on their color and therefore will fail on the test dataset.

## Setup environment

In [386]:
# basic modules
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from PIL import Image

# pytorch modules
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets.utils as dataset_utils
from torch.autograd import grad
from torch.nn import functional as F
from torchvision import datasets, models, transforms
from torchvision import __version__ as torchvision_version

In [387]:
# include plots in notebook
%matplotlib inline

In [388]:
# check environment
print("Python version: ", sys.version)
print("Pytorch version: ", torch.__version__)
print("Torchvision version: ", torchvision_version)

!nvidia-smi

Python version:  3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 06:04:10) 
[GCC 10.3.0]
Pytorch version:  1.13.0a0+d0d6b1f
Torchvision version:  0.14.0a0


## Prepare dataset

In [ ]:
def color_grayscale_arr(arr, green=True):
  """
  Convert grayscale MNIST images to either red or green MNIST images by expanding
  the image to three RGB dimensions. The grayscale image either gets applied to
  the red or green channel.
  """
  assert arr.ndim == 2
  
  dtype = arr.dtype
  h, w = arr.shape
  arr = np.reshape(arr, [h, w, 1])
  
  if green:
    arr = np.concatenate([np.zeros((h, w, 1), dtype=dtype),
                          arr,
                          np.zeros((h, w, 1), dtype=dtype)], axis=2)
  else:
    arr = np.concatenate([arr,
                          np.zeros((h, w, 2), dtype=dtype)],axis=2)
  return arr


class ColoredMNIST(datasets.VisionDataset):
  """
  Downloads the grayscale MNIST dataset and transforms it into a colored MNIST dataset.
  Digits smaller than 5 are colored red for the train and validation set. Numbers larger
  than 5 are colored green for the train and validation set. The colors of the digits have
  a 50% probability to be flipped.
  """
  def __init__(self, root='./data', env='train', transform=None, target_transform=None):
    super(ColoredMNIST, self).__init__(root, transform=transform,
                                target_transform=target_transform)

    self.prepare_colored_mnist()
    if env in ['train', 'val', 'test']:
      self.data_label_tuples = torch.load(os.path.join(self.root, 'ColoredMNIST', env) + '.pt')
    else:
      raise RuntimeError(f'{env} env unknown. Valid envs are train, val and test')

  def __getitem__(self, index):
    """
    Overriden method from datasets.VisionDataset to apply transformations to the data
    before providing them to the dataloader.
    """
    img, asdf, target = self.data_label_tuples[index]

    if self.transform is not None:
      img = self.transform(img)

    if self.target_transform is not None:
      target = self.target_transform(target)

    return img, asdf, target

  def __len__(self):
    return len(self.data_label_tuples)

  def prepare_colored_mnist(self):
    """
    Download 60'000 grayscale MNIST images, split them into a train, validation and a test
    dataset and transform them to a colored MNIST dataset.
    """
    
    def mnist_grayscale_to_color():
      def conversion_progress(idx, datasource, phase='train'):
        if idx % 5000 == 0:
            print(f'Converting {phase} image {idx}/{len(datasource)}')
      
      # http://yann.lecun.com/exdb/mnist/
      print('Preparing Colored MNIST')
      train_mnist = datasets.mnist.MNIST(self.root, train=True, download=True) # 60'000 samples for training
      test_mnist = datasets.mnist.MNIST(self.root, train=False, download=True) # 10'000 samples for validation and test

      train_set = []
      validation_set = []
      test_set = []
      
      for dataset in ['train_ds', 'test_ds']:
        datasource = train_mnist if (dataset == 'train_ds') else test_mnist

        for idx, (im, label) in enumerate(datasource):
          # determine train, validation, test phase/split
          if dataset == 'train_ds':
            phase = 'train'
          elif dataset == 'test_ds':
            if idx < len(datasource)//2:
              phase = 'validation'
            else:
              phase = 'test'
          
          # progress bar
          conversion_progress(idx, datasource, phase)
                
          # Assign binary digit label for small and large numbers
          true_label = 1 if label > 4 else 0

          # Assign random color labels to test set
          if phase == 'test':
            color_label = 1 if np.random.rand() < .5 else 0
          else:
            color_label = true_label
          
          # Color the digit
          colored_arr = color_grayscale_arr(np.array(im), green=color_label)
          
          # create dataset with image, true_label (1 for high numbers, 0 for low numbers) and color_label (potentially mixed up label for test dataset)
          if phase == 'train':
            train_set.append((Image.fromarray(colored_arr), true_label, color_label))
          elif phase == 'validation':
            validation_set.append((Image.fromarray(colored_arr), true_label, color_label))
          else:
            test_set.append((Image.fromarray(colored_arr), true_label, color_label))
            
      return train_set, validation_set, test_set
      
    
    colored_mnist_dir = os.path.join(self.root, 'ColoredMNIST')
    if os.path.exists(os.path.join(colored_mnist_dir, 'train.pt')) \
        and os.path.exists(os.path.join(colored_mnist_dir, 'val.pt')) \
        and os.path.exists(os.path.join(colored_mnist_dir, 'test.pt')):
      print('Colored MNIST dataset already exists')
      return
    
    train_set, val_set, test_set = mnist_grayscale_to_color()

    os.makedirs(colored_mnist_dir, exist_ok=True)
    torch.save(train_set, os.path.join(colored_mnist_dir, 'train.pt'))
    torch.save(val_set, os.path.join(colored_mnist_dir, 'val.pt'))
    torch.save(test_set, os.path.join(colored_mnist_dir, 'test.pt'))


In [ ]:
def plot_digits(dataset):
    """
    Plots some digits from a provided colored MNIST dataset to be analyzed.
    """
    fig = plt.figure(figsize=(13,8))
    columns = 6
    rows = 3
    ax = []
    
    for i in range(columns*rows):
        img, true_label, color_label = dataset[i]
        ax.append(fig.add_subplot(rows, columns, i + 1))
        ax[-1].set_title("True label: " + str(true_label) + 
                         "\nColor label: " + str(color_label) +
                         "\nFlipped: " + str(true_label != color_label))
        plt.imshow(np.transpose(img.cpu().numpy(), (1,2,0)))
        
    plt.tight_layout()
    plt.show()

In [ ]:
train_set = ColoredMNIST(root='./data',
                          env='train',
                          transform= transforms.Compose([transforms.ToTensor()]))
                            #   [transforms.ToTensor(),
                            #    transforms.Normalize(mean=(0.5,), std=(0.5,))]))
                            
plot_digits(train_set)

In [ ]:
val_set = ColoredMNIST(root='./data',
                          env='val',
                          transform= transforms.Compose([transforms.ToTensor()]))
                            #   [transforms.ToTensor(),
                            #    transforms.Normalize(mean=(0.5,), std=(0.5,))]))
                            
plot_digits(val_set)

In [ ]:
test_set = ColoredMNIST(root='./data',
                          env='test',
                          transform= transforms.Compose([transforms.ToTensor()]))
                            #   [transforms.ToTensor(),
                            #    transforms.Normalize(mean=(0.5,), std=(0.5,))]))
                            
plot_digits(test_set)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=10)

val_loader = torch.utils.data.DataLoader(dataset=val_set,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=10)

test_loader = torch.utils.data.DataLoader(dataset=test_set,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=10)

## Set device (For number crunching)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## Initialize network

In [ ]:
# Resnet50 transfer learning
# https://discuss.pytorch.org/t/how-to-modify-the-final-fc-layer-based-on-the-torch-model/766/2
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
# replace the last fully-connected layer
# parameters of newly constructed modules have required_grad=True by default
model.fc = nn.Linear(2048, 2)
model.to(device)
model.fc.weight # initialized with random numbers - Requires grad is true

## Train network

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=3):
    """
    Training procedure and performance measurement of the model.
    """
    history = {'train'          : {'loss' : [],
                                    'acc' : []},
                'validation'    : {'loss' : [],
                                    'acc' : []},
                'test'          : {'loss' : [],
                                    'acc' : []}}
        
    # epoch
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # phase
        for phase in ['train', 'validation', 'test']:
            if phase == 'train':
                model.train()
                datasource = train_loader
            elif phase == "validation":
                model.eval()
                datasource = val_loader
            else:
                model.eval()
                datasource = test_loader

            epoch_loss = 0.0
            running_corrects = 0
            label_counter = 0

            # batch
            for inputs, true_lables, _ in datasource:
                inputs = inputs.to(device)
                true_lables = true_lables.to(device)
                
                logits = model(inputs)
                preds = logits.argmax(dim=1)
                running_corrects += torch.sum(preds == true_lables.data)
                label_counter += true_lables.size()[0]
                
                batch_loss = criterion(logits, true_lables)
                epoch_loss += batch_loss.item()

                if phase == 'train':
                    optimizer.zero_grad()   # Sets the gradients of all optimized torch.Tensor to zero.
                    batch_loss.backward()   # compute gradients
                    optimizer.step()        # Performs a single optimization step (parameter update).
                
            epoch_acc = running_corrects.double() / label_counter
            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
            
            history[phase]['loss'].append(epoch_loss)
            history[phase]['acc'].append(epoch_acc.cpu())
            
    return model, history, inputs, true_lables, logits, batch_loss, label_counter

In [ ]:
# F.sigmoid (Map values between 0 and 1) + F.binary_cross_entropy
# https://zhang-yang.medium.com/how-is-pytorchs-binary-cross-entropy-with-logits-function-related-to-sigmoid-and-d3bd8fb080e7
# https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a
#criterion = F.binary_cross_entropy_with_logits #(input, target)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=.0001)

In [ ]:
model, history, inputs, labels, preds, batch_loss, label_counter = train_model(model, criterion, optimizer, num_epochs=20)

## Assess performance

In [ ]:
def plot_history(hist):
    """
    Plot the losses and accuracies during the training, validation and test procedures.
    """
    plt.subplot(1,2,1)
    plt.semilogy(range(len(hist['train']['loss'])), hist['train']['loss'], label='Train')
    plt.semilogy(range(len(hist['validation']['loss'])), hist['validation']['loss'], label='Validation')
    plt.semilogy(range(len(hist['test']['loss'])), hist['test']['loss'], label='Test')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()

    plt.subplot(1,2,2)
    plt.plot(range(len(hist['train']['acc'])), hist['train']['acc'], label='Train')
    plt.plot(range(len(hist['validation']['acc'])), hist['validation']['acc'], label='Validation')
    plt.plot(range(len(hist['test']['acc'])), hist['test']['acc'], label='Test')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy in %')
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()
    
plot_history(history)

This happens when you use Dropout, since the behaviour when training and testing are different.

When training, a percentage of the features are set to zero (50% in your case since you are using Dropout(0.5)). When testing, all features are used (and are scaled appropriately). So the model at test time is more robust - and can lead to higher testing accuracies.

In [ ]:
def random_tests(dataset):
    """
    Test and plot some digits from a provided colored MNIST dataset to be analyzed.
    """
    fig = plt.figure(figsize=(25,25))
    columns = 4
    rows = 8
    ax = []
    
    img, true_label, color_label = next(iter(dataset))
    model.eval()
    logits = model(img.to(device))
    pred = logits.argmax(dim=1)

    print("Batch accuracy: " + str(100 * torch.sum(pred == true_label.to(device)).item() / true_label.shape[0]) + "%")

    for i in range(columns*rows):
        ax.append(fig.add_subplot(8, 4, i + 1))
        if (pred[i].item() == true_label[i].item()):
            ax[-1].set_title("Ground truth: " + 
                             str(true_label[i].item()) + 
                             "\nPrediction: " + 
                             str(np.round(pred[i].item())) + 
                             "\nCorrect!")
        else:
            ax[-1].set_title("Ground truth: " + 
                             str(true_label[i].item()) + 
                             "\nPrediction: " + 
                             str(np.round(pred[i].item())) + 
                             "\nFooled!")
        plt.imshow(np.transpose(img[i].cpu().numpy(), (1,2,0)))
        
    plt.tight_layout()
    plt.show()

In [ ]:
random_tests(train_loader)

In [ ]:
random_tests(val_loader)

In [ ]:
random_tests(test_loader)